## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.37,-73.41,35.01,80,40,3.36,scattered clouds
1,1,Key West,US,24.56,-81.78,82.40,74,1,10.29,clear sky
2,2,Hobart,AU,-42.88,147.33,66.00,67,75,9.17,shower rain
3,3,Panambi,BR,-28.29,-53.50,58.55,73,0,15.39,clear sky
4,4,Ushuaia,AR,-54.80,-68.30,48.20,87,75,6.93,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df_TEST = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df_TEST.count()

City_ID                230
City                   230
Country                229
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df_TEST.dropna()
preferred_cities_df.count()

City_ID                229
City                   229
Country                229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Key West,US,82.40,clear sky,24.56,-81.78,
9,Carnarvon,AU,73.40,overcast clouds,-24.87,113.63,
15,San-Pedro,CI,76.89,clear sky,4.75,-6.64,
17,Kupang,ID,89.83,clear sky,-10.17,123.58,
20,Gladstone,AU,84.20,broken clouds,-23.85,151.25,
21,Georgetown,MY,78.80,scattered clouds,5.41,100.34,
22,Kenda,IN,75.20,haze,23.20,86.53,
26,Rikitea,PF,76.08,clear sky,-23.12,-134.97,
27,Kendari,ID,85.28,overcast clouds,-3.94,122.50,
32,Labuhan,ID,76.68,overcast clouds,-6.88,112.21,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Key West,US,82.40,clear sky,24.56,-81.78,Fairfield Inn & Suites by Marriott Key West
9,Carnarvon,AU,73.40,overcast clouds,-24.87,113.63,Hospitality Carnarvon
15,San-Pedro,CI,76.89,clear sky,4.75,-6.64,Hotel Maria
17,Kupang,ID,89.83,clear sky,-10.17,123.58,Sylvia Hotel Premier Kupang
20,Gladstone,AU,84.20,broken clouds,-23.85,151.25,Highpoint International
21,Georgetown,MY,78.80,scattered clouds,5.41,100.34,Cititel Penang
22,Kenda,IN,75.20,haze,23.20,86.53,Gouranga Hindu Hotel
26,Rikitea,PF,76.08,clear sky,-23.12,-134.97,Pension Maro'i
27,Kendari,ID,85.28,overcast clouds,-3.94,122.50,Hotel Wixel
32,Labuhan,ID,76.68,overcast clouds,-6.88,112.21,bador sepret


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))